In [ ]:
from keras.applications import inception_v3
from keras import backend as K
from keras.applications.imagenet_utils import decode_predictions
from keras.preprocessing import image
import numpy as np
K.set_learning_phase(0)

model = inception_v3.InceptionV3(weights='imagenet',include_top=True)

In [ ]:
for layer in model.layers:
    #if layer.name.startswith("activation_"):
    print (layer.name)

In [ ]:
np.array(model.layers[len(model.layers)-1].get_weights()[0]).shape

In [ ]:
activation_layers = [ layer.output for layer in model.layers if layer.name.startswith("activation_")]

In [ ]:
layer_names = [ layer.name for layer in model.layers if layer.name.startswith("activation_")]
layer_names

In [ ]:
from keras.models import Model
activation_model = Model(inputs=model.input, outputs=activation_layers)

In [ ]:
def preprocess_image(image_path):
    img = image.load_img(image_path)
    img = image.img_to_array(img)
    #convert single image to a batch with 1 image
    img = np.expand_dims(img, axis=0)    
    img = inception_v3.preprocess_input(img)
    return img

base_image_path = 'original_images/labrador.jpg'
# Load the image into a Numpy array
img = preprocess_image(base_image_path)

In [ ]:
activations = activation_model.predict(img)

In [ ]:
print(len(activation_layers))
activations[93].shape


In [ ]:
import matplotlib.pyplot as plt
#plt.matshow(activations[7][0, :, :, 0], cmap='viridis')
#plt.show()

In [ ]:
print("Number of Layers", len(activations))

In [ ]:
layer_names[46]

In [ ]:
images_per_row = 16
idx = 80
# Now let's display our feature maps
for layer_activation in activations[idx:idx+1]:
    # This is the number of features in the feature map
    n_features = layer_activation.shape[-1]
    # The feature map has shape (1, size1, size2, n_features)
    r = layer_activation.shape[1]
    c = layer_activation.shape[2]
    
    # We will tile the activation channels in this matrix
    n_cols = n_features // images_per_row
    display_grid = np.zeros((r * n_cols, images_per_row * c))
    print(display_grid.shape)
    # We'll tile each filter into this big horizontal grid
    for col in range(n_cols):
        for row in range(images_per_row):
            channel_image = layer_activation[0,
            :, :,
            col * images_per_row + row]
            # Post-process the feature to make it visually palatable
            channel_image -= channel_image.mean()
            channel_image /= channel_image.std()
            channel_image *= 64
            channel_image += 128
            channel_image = np.clip(channel_image, 0, 255).astype('uint8')
            display_grid[col * r : (col + 1) * r,
            row * c : (row + 1) * c] = channel_image
    # Display the grid
    scale = 1. / r
    plt.figure(figsize=(scale * display_grid.shape[1],
    scale * display_grid.shape[0]))
    plt.title(layer_names[idx]+" #filters="+str(n_features))
    plt.grid(False)
    plt.imshow(display_grid, aspect='auto', cmap='viridis')


In [ ]:
images_per_row = 8
idx = 1
# Now let's display our feature maps
layer_activation=activations[idx]
# This is the number of features in the feature map
n_features = layer_activation.shape[-1]
# The feature map has shape (1, size1, size2, n_features)
r = layer_activation.shape[1]
c = layer_activation.shape[2]
    
# We will tile the activation channels in this matrix
n_cols = n_features // images_per_row
display_grid = np.zeros((r * n_cols, images_per_row * c))
print(display_grid.shape)


In [ ]:
plt.show()

## CNN Visualization by Gradient Accent

In [ ]:
layer_name = 'activation_94'
filter_index = 0
layer_output = model.get_layer(layer_name).output
loss = K.mean(layer_output[:, :, :, filter_index])

In [ ]:
grads = K.gradients(loss, model.input)[0]
# We add 1e-5 before dividing so as to avoid accidentally dividing by 0.
grads /= (K.sqrt(K.mean(K.square(grads))) + 1e-5)

In [ ]:
iterate = K.function([model.input], [loss, grads])
# Let's test it:
import numpy as np
loss_value, grads_value = iterate([np.zeros((1, 150, 150, 3))])

In [ ]:
# We start from a gray image with some noise
input_img_data = np.random.random((1, 150, 150, 3)) * 20 + 128.
# Run gradient ascent for 40 steps
step = 1. # this is the magnitude of each gradient update
for i in range(40):
    # Compute the loss value and gradient value
    loss_value, grads_value = iterate([input_img_data])
    # Here we adjust the input image in the direction that maximizes the loss
    input_img_data += grads_value * step

In [ ]:
def deprocess_image(x):
    # normalize tensor: center on 0., ensure std is 0.1
    x -= x.mean()
    x /= (x.std() + 1e-5)
    x *= 0.1
    # clip to [0, 1]
    x += 0.5
    x = np.clip(x, 0, 1)
    # convert to RGB array
    x *= 255
    x = np.clip(x, 0, 255).astype('uint8')
    return x

In [ ]:
def generate_pattern(layer_name, filter_index, size=150):
    # Build a loss function that maximizes the activation
    # of the nth filter of the layer considered.
    layer_output = model.get_layer(layer_name).output
    loss = K.mean(layer_output[:, :, :, filter_index])
    # Compute the gradient of the input picture wrt this loss
    grads = K.gradients(loss, model.input)[0]
    # Normalization trick: we normalize the gradient
    grads /= (K.sqrt(K.mean(K.square(grads))) + 1e-5)
    # This function returns the loss and grads given the input picture
    iterate = K.function([model.input], [loss, grads])
    # We start from a gray image with some noise
    input_img_data = np.random.random((1, size, size, 3)) * 20 + 128.
    # Run gradient ascent for 40 steps
    step = 2.
    for i in range(80):
        loss_value, grads_value = iterate([input_img_data])
        input_img_data += grads_value * step
        img = input_img_data[0]
    return deprocess_image(img)

In [ ]:
plt.imshow(generate_pattern('activation_2', 1))

In [ ]:
plt.show()

In [ ]:
idx = 0
layer_name = layer_names[idx]
images_per_row = 8
size = 64
margin = 5

n_features = activation_layers[idx].shape[-1]
n_cols = n_features // images_per_row

In [ ]:
n_features, n_cols

In [ ]:

# This a empty (black) image where we will store our results.
results = np.zeros((images_per_row * size + 7 * margin, images_per_row * size + 7 * margin, 3))
for i in range(n_cols): # iterate over the rows of our results grid
    for j in range(images_per_row): # iterate over the columns of our results grid
    # Generate the pattern for filter `i + (j * 8)` in `layer_name`
        filter_img = generate_pattern(layer_name, (i * images_per_row) +j, size=size)
        # Put the result in the square `(i, j)` of the results grid
        horizontal_start = i * size + i * margin
        horizontal_end = horizontal_start + size
        vertical_start = j * size + j * margin
        vertical_end = vertical_start + size
        results[horizontal_start: horizontal_end, vertical_start: vertical_end, :] = filter_img


In [ ]:
# Display the results grid
plt.figure(figsize=(20, 20))
plt.title(layer_names[idx]+" #filters="+str(n_features))
plt.imshow(results)
plt.show()

## Deep Dream

In [ ]:
layer_name = 'activation_41'
activation = model.get_layer(layer_name).output

# We avoid border artifacts by only involving non-border pixels in the loss.
scaling = K.prod(K.cast(K.shape(activation), 'float32'))
loss = K.sum(K.square(activation[:, 2: -2, 2: -2, :])) / scaling

# This holds our generated image
dream = model.input

# Compute the gradients of the dream with regard to the loss.
grads = K.gradients(loss, dream)[0]

# Normalize gradients.
grads /= K.maximum(K.mean(K.abs(grads)), 1e-7)


iterate_grad_ac_step = K.function([dream], [loss, grads])

In [ ]:
def gradient_ascent(x, iterations, step, max_loss=None):
    for i in range(iterations):
        loss_value, grad_values = iterate_grad_ac_step([x])
        print('...Loss value at', i, ':', loss_value)
        if max_loss is not None and loss_value > max_loss:
            break        
        x += step * grad_values
    return x

In [ ]:
import scipy

def deprocess_image(x):
    # Util function to convert a tensor into a valid image.
    if K.image_data_format() == 'channels_first':
        x = x.reshape((3, x.shape[2], x.shape[3]))
        x = x.transpose((1, 2, 0))
    else:
        x = x.reshape((x.shape[1], x.shape[2], 3))
    x /= 2.
    x += 0.5
    x *= 255.
    x = np.clip(x, 0, 255).astype('uint8')
    return x

def resize_img(img, size):
    img = np.copy(img)
    factors = (1,
               float(size[0]) / img.shape[1],
               float(size[1]) / img.shape[2],
               1)
    return scipy.ndimage.zoom(img, factors, order=1)


def save_img(img, fname):
    pil_img = deprocess_image(np.copy(img))
    scipy.misc. (fname, pil_img)


In [ ]:
num_octave = 4 # Number of scales at which to run gradient ascent
octave_scale = 1.4 # Size ratio between scales
iterations = 20 # Number of ascent steps per scale

# If our loss gets larger than 10, 
# we will interrupt the gradient ascent process, to avoid ugly artifacts
max_loss = 20.

base_image_path = 'original_images/blue-sky-bright-clouds-125458.jpg'
# Load the image into a Numpy array
img = preprocess_image(base_image_path)
print(img.shape)
# We prepare a list of shape tuples
# defining the different scales at which we will run gradient ascent
original_shape = img.shape[1:3]
successive_shapes = [original_shape]
for i in range(1, num_octave):
    shape = tuple([int(dim / (octave_scale ** i)) for dim in original_shape])
    successive_shapes.append(shape)

# Reverse list of shapes, so that they are in increasing order
successive_shapes = successive_shapes[::-1]

# Resize the Numpy array of the image to our smallest scale
original_img = np.copy(img)
shrunk_original_img = resize_img(img, successive_shapes[0])

print(successive_shapes)

In [ ]:
MAX_ITRN = 20
MAX_LOSS = 20
learning_rate = 0.01

for shape in successive_shapes:
    print('Processing image shape', shape)
    img = resize_img(img, shape)
    img = gradient_ascent(img,
                          iterations=MAX_ITRN,
                          step=learning_rate,
                          max_loss=MAX_LOSS)
    upscaled_shrunk_original_img = resize_img(shrunk_original_img, shape)
    same_size_original = resize_img(original_img, shape)
    lost_detail = same_size_original - upscaled_shrunk_original_img
    print('adding lost details', lost_detail.shape)
    img += lost_detail
    shrunk_original_img = resize_img(original_img, shape)
    save_img(img, fname='dream_at_scale_' + str(shape) + '.png')

save_img(img, fname='final_dream.png')


## Heatmaps of class activation

In [ ]:
base_image_path = 'original_images/labrador.jpg'
# Load the image into a Numpy array
img = preprocess_image(base_image_path)

In [ ]:
preds = model.predict(img)
print('Predicted:', decode_predictions(preds, top=3)[0])

In [ ]:
np.argmax(preds[0])

In [ ]:
# This is the "african elephant" entry in the prediction vector
dog_output = model.output[:, 208]
# The is the output feature map of the `block5_conv3` layer,
# the last convolutional layer in VGG16
last_conv_layer = model.get_layer('activation_94')
# This is the gradient of the "dog" class with regard to
# the output feature map of `activation_94`
grads = K.gradients(dog_output, last_conv_layer.output)[0]